In [1]:
import ccxt
import pandas as pd
import numpy as np
import time

In [ ]:
exchange = ccxt.binance({
    'enableRateLimit': True})

In [3]:
def fetch_price(symbol='BTC/USDT'):
    df = exchange.fetch_ticker(symbol)
    return df['last']

In [4]:
def trading_logic(symbol='BTC/USDT', window=20, z_score_threshold=2.0):
    while True:
        # Fetch recent price data
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1m', limit=window)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['returns'] = df['close'].pct_change().dropna()

        ##calculate z-score
        mean = df['returns'].mean()
        std = df['returns'].std()
        latest_return = df['returns'].iloc[-1]
        z_score = (latest_return - mean) / std if std != 0 else 0

        current_price = fetch_price(symbol)
        print(f"Current Price: {current_price:.2f}, Z-Score: {z_score: .2f}")

        ###Trading logic based on z-score
        if z_score > z_score_threshold:
            print("Sell signal: Price too high, reverting to mean")
        elif z_score < -z_score_threshold:
            print("Buy signal: Price too low, reverting to mean")
        else:
            print("Z-Score is within the threshold, no action taken.")
        time.sleep(60)

    

In [ ]:
if __name__ == "__main__":
    trading_logic()

Current Price: 108806.75, Z-Score:  0.05
Z-Score is within the threshold, no action taken.
